In [ ]:
#| default_exp src.rgxlog_interpreter.tests.test_symbol_table

In [ ]:
#| export
import pytest
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.state.symbol_table import SymbolTable
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.datatypes.primitive_types import DataTypes, DataTypeMapping, Span

In [ ]:
#| export
def test_variable_operations():
    table = SymbolTable()
    table.set_var_value_and_type("var1", 10, int)
    assert table.contains_variable("var1")
    assert table.get_variable_type("var1") == int
    assert table.get_variable_value("var1") == 10

def test_relation_operations():
    table = SymbolTable()
    table.add_relation_schema("relation1", [int, str], is_rule=False)
    assert table.contains_relation("relation1")
    assert table.get_relation_schema("relation1") == [int, str]

def test_variable_relation_interactions():
    table = SymbolTable()
    
    table.set_var_value_and_type("var1", 10, DataTypes.integer)
    assert table.contains_variable("var1")
    assert table.get_variable_type("var1") == DataTypes.integer
    assert table.get_variable_value("var1") == 10
    
    table.add_relation_schema("relation1", [DataTypes.integer, DataTypes.string], is_rule=False)
    assert table.contains_relation("relation1")
    assert table.get_relation_schema("relation1") == [DataTypes.integer, DataTypes.string]
    
    with pytest.raises(KeyError):
        table.get_variable_type("relation1")
    with pytest.raises(NameError):
        table.get_relation_schema("var1")

def test_ie_function_operations():
    def ie_function(x):
        return x + 1

    table = SymbolTable()
    table.register_ie_function(ie_function, "func1", [DataTypes.integer], [DataTypes.integer])
    
    assert table.contains_ie_function("func1")
    assert table.get_ie_func_data("func1").ie_function_def(5) == 6

    table.remove_ie_function("func1")
    assert not table.contains_ie_function("func1")

def test_rule_relation_operations():
    table = SymbolTable()
    
    table.add_relation_schema("rule_relation1", [DataTypes.string, DataTypes.integer], is_rule=True)
    assert table.contains_relation("rule_relation1")
    assert "rule_relation1" in [relation for relation, _ in table.get_all_relations()]

    table.remove_rule_relation("rule_relation1")
    assert not table.contains_relation("rule_relation1")
    assert "rule_relation1" not in [relation for relation, _ in table.get_all_relations()]

def test_all_relations_and_ie_functions():
    table = SymbolTable()
    
    table.add_relation_schema("relation1", [DataTypes.integer], is_rule=False)
    table.add_relation_schema("relation2", [DataTypes.string], is_rule=False)
    
    def ie_function(x):
        return x + 1

    table.register_ie_function(ie_function, "func1", [DataTypes.integer], [DataTypes.integer])
    table.register_ie_function(ie_function, "func2", [DataTypes.string], [DataTypes.string])
    
    all_relations = table.get_all_relations()
    assert ("relation1", [DataTypes.integer]) in all_relations
    assert ("relation2", [DataTypes.string]) in all_relations
    
    all_ie_functions = table.get_all_registered_ie_funcs()
    assert "func1" in all_ie_functions
    assert "func2" in all_ie_functions

def test_rule_relation_schema_check():
    table = SymbolTable()

    table.add_relation_schema("rule_relation1", [DataTypes.string, DataTypes.integer], is_rule=True)
    
    with pytest.raises(Exception):
        table.add_relation_schema("rule_relation1", [DataTypes.integer, DataTypes.string], is_rule=True)

def test_rule_relation_name_conflict():
    table = SymbolTable()

    table.add_relation_schema("relation1", [DataTypes.integer], is_rule=False)
    
    with pytest.raises(Exception):
        table.add_relation_schema("relation1", [DataTypes.string, DataTypes.integer], is_rule=True)

def test_remove_rule_relation_bug():
    table = SymbolTable()

    table.add_relation_schema("rule_relation1", [DataTypes.string, DataTypes.integer], is_rule=True)

    table.remove_rule_relation("rule_relation1")

    with pytest.raises(KeyError):
        table.remove_rule_relation("rule_relation1")